<a href="https://colab.research.google.com/github/MasterMilkX/BMO_chatbot_prototype/blob/main/bmo_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#scrapper

In [ ]:
#use the microstudio_scrapper to get links.txt

In [ ]:
# !pip install wget
import wget
import zipfile

In [ ]:
# find source 
def author_game(string):
    index = []
    for i, c in enumerate(string):
        if c == '/':
            index.append(i)
    return string[index[2] : index[4]]

# extract ms folder from the zip file
def extract_zip(file_name):
    archive = zipfile.ZipFile(file_name+'.zip')

    for file in archive.namelist():
        if file.startswith('ms'):
            archive.extract(file, 'codes/'+file_name)
        if file.startswith('sprites'):
            archive.extract(file, 'sprites/'+file_name)

In [ ]:
file1 = open('links.txt', 'r')
lines = file1.readlines()

In [ ]:
for line in lines:
    url = line
    url_name = "_".join(author_game(url).split("/")[1:])
    wget.download(url, url_name + '.zip')
    extract_zip(url_name)
    # os.remove(url_name + '.zip')

In [ ]:
!mkdir zips

mkdir: cannot create directory ‘zips’: File exists


In [ ]:
!mv sprites /content/drive/MyDrive/microstudio_games

In [ ]:
!du /content/drive/MyDrive/microstudio_games -h

# data

In [62]:
#path to zip file
!cp "/content/drive/MyDrive/bmo s/microstudio_games.zip" ./

In [63]:
import zipfile
with zipfile.ZipFile("microstudio_games.zip", 'r') as zip_ref:
    zip_ref.extractall("./")

In [64]:
#convert directories to list of source code
import os
def getListOfFiles(dirName):
    listOfFile = os.listdir(dirName)
    allFiles = list()
    for entry in listOfFile:
        fullPath = os.path.join(dirName, entry)
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles 
code_list = getListOfFiles("microstudio_games/codes")

In [65]:
# converting into pandas df/csv
import pandas as pd
df = pd.DataFrame(columns = ["codes"])
for file in code_list:
  with open(file, 'r') as f:
    string = f.read()
    df = df.append({"codes" : string}, ignore_index=True)

df.to_csv('data.csv')

# modify data

In [ ]:
data = df
data

In [67]:
data_m_lines = data.copy()
for i, code in enumerate(data["codes"]):
  data_m_lines["codes"][i] = "%"+ code.replace("\n", "\n%")

data_m_lines.to_csv("data_m_lines.csv")

In [68]:
import pandas as pd
import numpy as np

df = pd.read_csv('data_m_lines.csv')
df['split'] = np.random.randn(df.shape[0], 1)

msk = np.random.rand(len(df)) <= 0.8

train = df[msk]
val = df[~msk]
train['codes'].to_csv("train.csv", index=False)
val['codes'].to_csv("val.csv", index = False)

# run using hugging face examples

In [23]:
with open('run_clm.py', 'w') as f:
  import requests
  r = requests.get("https://raw.githubusercontent.com/huggingface/transformers/main/examples/pytorch/language-modeling/run_clm.py")
  f.write(r.text)

In [ ]:
!pip install evaluate
!pip install git+https://github.com/huggingface/transformers

In [6]:
!python run_clm.py \
    --model_name_or_path "Salesforce/codegen-350M-multi" \
    --train_file /content/train.csv \
    --validation_file /content/val.csv \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --do_train yes \
    --do_eval yes \
    --output_dir /outputs/test-clm

INFO:__main__:Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
include_inputs_for_me

## results...?

In [40]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-350M-multi")
model2 = AutoModelForCausalLM.from_pretrained("Salesforce/codegen-350M-multi")
model = torch.load('/outputs/test-clm/pytorch_model.bin')

In [35]:
!du /outputs -h

4.1G	/outputs/test-clm/checkpoint-1500
4.1G	/outputs/test-clm/checkpoint-1000
4.1G	/outputs/test-clm/checkpoint-3000
4.1G	/outputs/test-clm/checkpoint-500
12K	/outputs/test-clm/runs/Oct23_15-01-10_e0ad22031bf2/1666537286.063863
28K	/outputs/test-clm/runs/Oct23_15-01-10_e0ad22031bf2
32K	/outputs/test-clm/runs
4.1G	/outputs/test-clm/checkpoint-2000
4.1G	/outputs/test-clm/checkpoint-2500
26G	/outputs/test-clm
26G	/outputs


In [ ]:
model2.resize_token_embeddings(50295)
model2.state_dict()

In [46]:
# model2 = model2.resize_token_embeddings(50295)
model2.load_state_dict(model, strict=False)

<All keys matched successfully>

In [49]:
def get_output(text): #text = "%draw = function():"
  input_ids = tokenizer(text, return_tensors="pt").input_ids

  generated_ids = model2.generate(input_ids, max_length=128)
  print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))

In [50]:
get_output("%movements = function()")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


%movements = function()
%  if cash > 0 then
%    if direction == 1 then
%      x += speed * cash
%    elsif direction == -1 then
%      x -= speed * cash
%    end
%  end
%  
%  if cash < 0 then
%    if direction == 1 then
%      x -= speed
%    elsif direction == -1 then
%      x += speed
%    end
%  end
%  
%  if cash > 0 then
%    if direction == 1 then
%      y


In [51]:
get_output("%move_player = function()")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


%move_player = function()
%  if is_taking_turn(player) then 
%    is_taking_turn = false
%    take_turn = true
%    turning_left = false
%    turning_right = false
%    
%    local player_x = player.x
%    local player_y = player.y
%    
%    // if player is moving left, check if there is any obstacles on the left
%    if player.vx < 0 and 
%      (map.get_block(player_


In [52]:
get_output("% //flip the sprite")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


% //flip the sprite if facing left
%      if o.x<0 then
%        o.vx = -o.vx
%        o.x = -o.x
%        o.hflip = -1
%      elsif o.x>0 then
%        o.vx =  o.vx
%        o.x = -o.x
%        o.hflip = 1
%      end
%    end
%  end
%end
%
%// sets the length of a movement vector to 1


In [55]:
get_output("% if onLadder then \n%  //climb ladder")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


% if onLadder then 
%  //climb ladder
%  //Add upper/lower link
%  //Go to ladder rest
%  //Climbing state
%  if not onLadder.isTop then
%    if not onLadder.isOpen then
%      if not onLadder.isBottom then
%        if not onLadder.isTop then
%          onLadder = onLadder.clone()
%          onLadder.setTop(false)
%        end
%      else
%        onLadder.set


In [56]:
get_output("% //show text on screen")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


% //show text on screen
%  if show_timer then
%    // draw timer
%    if timer<60 then
%      screen.setAlpha(0.4)
%      screen.fillRect(95, 90, 100, 20, "#000")
%      screen.setAlpha(1)
%      if show_timer<10 then
%        screen.drawText("h:mm:ss.0000", 100, 90, 10, "#FFF")
%      else
%        screen.drawText("h:mm:ss.000000", 100, 90


In [57]:
get_output("% //print 'BMO chatbot' on screen")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


% //print 'BMO chatbot' on screen
%    if keyboard.press.SPACE or mouse.press or touch.press then
%      if (not status) then
%        status = "movel"
%        if random.next()<0.005 then
%          if mouse.x<-40 and mouse.x>=-40 and mouse.y<-80 and mouse.y>=-80 then
%            status = "run"
%          end
%        end
%      end
%    end
%    
%    if status == "run


In [60]:
get_output("% name = \"bmo says hi\"\n% //show name on top left")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


% name = "bmo says hi"
% //show name on top left
%  h = 80
%  x = 170
%  y = -80
%  l = 50
%  w = 170
%  h = 40
%  bg ="rgb(0,0,0)"
%  text = "microStudio Jam #3"
%  text_size = 15
%  score = 0
%  highscore = storage.get(high)
%  if(highscore) then
%    highscore = highscore+1
%


In [61]:
get_output("%//show name on top left")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


%//show name on top left
%  if global.show_name then
%    screen.drawText(name,0,global.SW-10,9,"rgb(255,255,255)")
%  end
%  
%  //if invert then
%    screen.drawSprite("invert",0,0,width,height)
%  //end
%  
%  //if not show_controls then
%    screen.drawSprite("touchcontrols",0,global.SW-10,12,12)
%


In [82]:
get_output("%//show name on top right")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


%//show name on top right
%  if global.show_name then
%    screen.drawTextOutline("microStudio Jam #3", 0, -55, 21,"rgb(255,0,0)")
%    screen.drawText("microStudio Jam #3", 0, -55, 20, "#FFF")
%    screen.drawTextOutline("by MGB", 0, -64, 52,"rgb(255,0,0)")
%    screen.drawText("by MGB", 0, -64, 50, "#FFF


# trying something with the comments

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("data.csv", index_col=False)
df['codes']

0       if start then\n  credits = {}\n  \n  \n  credi...
1       if start then\n  map = {}\n  \n  \n  map.name ...
2       if start then\n  player = {}\n  \n  \n  player...
3       function init()\n  screen:setPixelated("pixela...
4       function string:split(sep)\n  local sep, field...
                              ...                        
1232    /*\n  All of the code here is the level design...
1233    mower=object\n  \n  init=function()\n    ACC=0...
1234    sfx=object\n  \n  init=function()\n    sfxvol=...
1235    /*\n  Level section - Draws the 4x4 preview le...
1236    game=object\n  \n  init=function()\n    MAPXOF...
Name: codes, Length: 1237, dtype: object

In [ ]:
def add_comment_code(string, spans):
  coms = [string[a:b] for (a,b) in spans]
  codes = []

  for i in range(len(spans)):
    start = spans[i][1]
    if(i==len(spans)-1):
      end = -1
    else:
      end = spans[i+1][0]
    codes.append(string[start:end])

  coms = [com.replace("//", " ").replace("\n"," ") for com in coms]
  
  return coms, codes

In [ ]:
import re

# comment_block , all code before next comment
l_comments = []
l_codes = []

for string in df['codes']:
  if(type(string) != str):
    continue
  
  l_spans = []
  prev = -1

  for i in re.finditer(r'\s*//.*\n',string):
    if(prev == i.span()[0]):
      temp = l_spans[-1][0]
      del l_spans[-1]
      l_spans.append((temp, i.span()[1]))
    else:
      l_spans.append((i.span()[0],i.span()[1]))
    prev = i.span()[1]

  com, cod = add_comment_code(string, l_spans)
  l_comments += com
  l_codes += cod

In [ ]:
df = pd.DataFrame(columns= ["comments", "codes"], data = [[l_comments[i], l_codes[i]] for i in range(len(l_comments))])

In [ ]:
df[df["comments"].str.contains('door')][df["comments"].str.contains('open')]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,comments,codes
3,lets climb tht ladder,hero.fall = 0\n her...
13,lets climb tht ladder,e.fall = 0\n e....
58,lets climb tht ladder,hero.fall = 0\n her...
72,lets climb tht ladder,e.fall = 0\n e....
92,lets climb tht ladder,e.fall = 0\n e....
95,lets climb tht ladder,e.fall = 0\n e.x = ...


# copy paste train codegen by salesforce
 do not run, does not work

In [ ]:
!pip install --upgrade pip setuptools
!pip install torch --extra-index-url https://download.pytorch.org/whl/cu113
!pip install transformers==4.21.1 datasets==1.16.1 deepspeed==0.7.0
!pip install mpi4py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 36.7 MB/s 
     |████████████████████████████████| 1.2 MB 52.7 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu113
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.3/298.3 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.0/630.0 kB 36.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 14.8 MB/s eta 0:00:

In [ ]:
!deepspeed --num_gpus=1 train_deepspeed.py

[2022-10-23 13:54:41,546] [WARNING] [runner.py:178:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
[2022-10-23 13:54:41,579] [INFO] [runner.py:504:main] cmd = /usr/bin/python3 -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMF19 --master_addr=127.0.0.1 --master_port=29500 train_deepspeed.py
[2022-10-23 13:54:43,679] [INFO] [launch.py:129:main] 0 NV_LIBNCCL_DEV_PACKAGE=libnccl-dev=2.8.4-1+cuda11.2
[2022-10-23 13:54:43,679] [INFO] [launch.py:129:main] 0 NCCL_VERSION=2.8.4-1
[2022-10-23 13:54:43,679] [INFO] [launch.py:129:main] 0 NV_LIBNCCL_PACKAGE_VERSION=2.8.4-1
[2022-10-23 13:54:43,679] [INFO] [launch.py:129:main] 0 NV_LIBNCCL_PACKAGE=libnccl2=2.8.4-1+cuda11.2
[2022-10-23 13:54:43,679] [INFO] [launch.py:129:main] 0 NV_LIBNCCL_DEV_PACKAGE_NAME=libnccl-dev
[2022-10-23 13:54:43,680] [INFO] [launch.py:129:main] 0 NV_LIBNCCL_PACKAGE_NAME=libnccl2
[2022-10-23 13:54:43,680] [INFO] [launch.py:129:main] 0 NV_LIBNCCL_DEV_PACKAGE_VER

In [ ]:
import os
import argparse
import random
import math
import mpi4py

from time import time

import numpy as np

import torch

from transformers import AutoConfig, AutoModelForCausalLM

import deepspeed

In [ ]:
DEEPSPEED_CONFIG = \
{
    'fp16': {'enabled': True, 'loss_scale': 0, 'loss_scale_window': 1000, 'initial_scale_power': 12, 'hysteresis': 2, 'min_loss_scale': 1},
    'optimizer': {'type': 'AdamW', 'params': {'lr': 1e-05, 'betas': [0.9, 0.999], 'eps': 1e-08, 'weight_decay': 0.0}},
    'scheduler': {'type': 'WarmupLR', 'params': {'warmup_min_lr': 0, 'warmup_max_lr': 1e-05, 'warmup_num_steps': 100}},
    'zero_optimization': {
        'stage': 3,
        'offload_optimizer': {'device': 'cpu', 'pin_memory': False},
        'offload_param': {'device': 'cpu', 'pin_memory': False},
        'overlap_comm': True,
        'contiguous_gradients': True,
        'sub_group_size': 1e9,
        'reduce_bucket_size': 16777216,
        'stage3_prefetch_bucket_size': 15099494.4,
        'stage3_param_persistence_threshold': 40960,
        'stage3_max_live_parameters': 1e9,
        'stage3_max_reuse_distance': 1e9,
        'stage3_gather_fp16_weights_on_model_save': True
    },
    'train_batch_size': 32,
    'train_micro_batch_size_per_gpu': 2,
    'gradient_accumulation_steps': 16,
    'gradient_clipping': 1.0,
    'steps_per_print': 8,
    'wall_clock_breakdown': False,
    'compression_training': {'weight_quantization': {'shared_parameters': {}, 'different_groups': {}}, 'activation_quantization': {'shared_parameters': {}, 'different_groups': {}}, 'sparse_pruning': {'shared_parameters': {}, 'different_groups': {}}, 'row_pruning': {'shared_parameters': {}, 'different_groups': {}}, 'head_pruning': {'shared_parameters': {}, 'different_groups': {}}, 'channel_pruning': {'shared_parameters': {}, 'different_groups': {}}}
}


def create_args(args=argparse.Namespace()):

    args.seed = 42

    args.model = 'Salesforce/codegen-350M-multi'

    args.deepspeed_config = DEEPSPEED_CONFIG

    args.opt_steps_train = 1000

    return args


In [ ]:
## train

def train(args):

    #######################
    ## preamble

    set_seed(args.seed)


    #######################
    ## model

    print('initializing model')

    config = AutoConfig.from_pretrained(args.model)
    config.gradient_checkpointing = True
    config.use_cache = False

    model = AutoModelForCausalLM.from_pretrained(args.model, config=config)

    model.train()
    # TODO(enijkamp): we need to set this flag twice?
    model.gradient_checkpointing_enable()


    #######################
    ## deepspeed

    print('initializing deepspeed')

    model_parameters = list(filter(lambda p: p.requires_grad, model.parameters()))
    model_engine, optimizer, _, _ = deepspeed.initialize(config=args.deepspeed_config, model=model, model_parameters=model_parameters)

    torch.cuda.empty_cache()


    #######################
    ## train

    print('starting training')

    input_ids = torch.randint(low=0, high=10, size=[args.deepspeed_config['train_micro_batch_size_per_gpu'], 1024], dtype=torch.int64).cuda()

    for step in range(args.opt_steps_train+1):

        loss = model_engine(input_ids=input_ids, labels=input_ids).loss

        model_engine.backward(loss)
        model_engine.step()

        print(f'{step} {loss:8.3f}')
    return model_engine

In [ ]:
########################################################################################################
## preamble

def set_gpus(gpu):
    torch.cuda.set_device(gpu)


def set_seed(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)


def set_cuda(deterministic=True):
    if torch.cuda.is_available():
        torch.backends.cudnn.deterministic = deterministic
        torch.backends.cudnn.benchmark = not deterministic


def get_exp_id(file):
    return os.path.splitext(os.path.basename(file))[0]


def get_output_dir(exp_id):
    import datetime
    t = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
    output_dir = os.path.join('output/' + exp_id, t)
    return output_dir


def copy_source(file, output_dir):
    import shutil
    shutil.copyfile(file, os.path.join(output_dir, os.path.basename(file)))

In [ ]:
# def main():
# data_s = "/content/data.csv"
data_s = "/content/data_m_lines.csv"
# preamble
exp_id = get_exp_id(data_s)
output_dir = get_output_dir(exp_id)

# args
args = create_args()
args.output_dir = output_dir
args.exp_id = exp_id

# output
os.makedirs(args.output_dir, exist_ok=True)
copy_source(data_s, args.output_dir)

# train
me = train(args=args)

initializing model


initializing deepspeed
[2022-10-23 13:55:21,868] [INFO] [logging.py:68:log_dist] [Rank -1] DeepSpeed info: version=0.7.0, git-hash=unknown, git-branch=unknown
[2022-10-23 13:55:21,874] [INFO] [comm.py:613:init_distributed] Not using the DeepSpeed or dist launchers, attempting to detect MPI environment...
[2022-10-23 13:55:22,368] [INFO] [comm.py:670:mpi_discovery] Discovered MPI settings of world_rank=0, local_rank=0, world_size=1, master_addr=172.28.0.2, master_port=29500
[2022-10-23 13:55:22,371] [INFO] [comm.py:630:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
[2022-10-23 13:55:22,382] [WARNING] [config_utils.py:64:_process_deprecated_field] Config parameter stage3_gather_fp16_weights_on_model_save is deprecated use gather_16bit_weights_on_model_save instead
[2022-10-23 13:55:27,220] [INFO] [logging.py:68:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled: False


/usr/local/lib/python3.7/dist-packages/_distutils_hack/__init__.py:19: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  "Distutils was imported before Setuptools, but importing Setuptools "
/usr/local/lib/python3.7/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Installed CUDA version 11.2 does not match the version torch was compiled with 11.3 but since the APIs are compatible, accepting this combination
Using /root/.cache/torch_extensions/py37_cu113 as PyTorch extensions root...
Creating extension directory /root/.cache/torch_extensions/py37_cu113/cpu_adam...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py37_cu113/cpu_adam/build.ninja...
Building extension module cpu_adam...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module cpu_adam...
Time to load cpu_adam op: 31.845327854156494 seconds
[2022-10-23 13:56:04,485] [INFO] [logging.py:68:log_dist] [Rank 0] Using DeepSpeed Optimizer param name adamw as basic optimizer
[2022-10-23 13:56:04,496] [INFO] [logging.py:68:log_dist] [Rank 0] DeepSpeed Basic Optimizer = {basic_optimizer.__class__.__name__}
[2022-10-23 13:56:04,506] [INFO] [utils.py:53:is_zero_supported

/usr/local/lib/python3.7/dist-packages/transformers/models/codegen/modeling_codegen.py:167: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at  ../aten/src/ATen/native/TensorCompare.cpp:402.)
  attn_weights = torch.where(causal_mask, attn_weights, mask_value)


0    3.805
1    3.805
2    3.805
3    3.805
4    3.805
5    3.805
6    3.805
[2022-10-23 13:56:43,243] [INFO] [timer.py:207:stop] 0/8, RunningAvgSamplesPerSec=1.5181804679522688, CurrSamplesPerSec=1.5049135777929032, MemAllocated=0.21GB, MaxMemAllocated=2.03GB
7    3.805
8    3.805
9    3.805
10    3.805
11    3.805
12    3.805
13    3.805
14    3.805
[2022-10-23 13:56:55,191] [INFO] [timer.py:207:stop] 0/16, RunningAvgSamplesPerSec=1.4110797731439624, CurrSamplesPerSec=0.7400870765617877, MemAllocated=0.11GB, MaxMemAllocated=2.03GB
15    3.805
16    3.027
17    3.027
18    3.027
19    3.027
20    3.027
21    3.027
22    3.027
[2022-10-23 13:57:05,841] [INFO] [timer.py:207:stop] 0/24, RunningAvgSamplesPerSec=1.443356124437662, CurrSamplesPerSec=1.5129302151098094, MemAllocated=0.21GB, MaxMemAllocated=2.03GB
23    3.027
24    3.027
25    3.027
26    3.027
27    3.027
28    3.027
29    3.027
30    3.027
[2022-10-23 13:57:17,764] [INFO] [timer.py:207:stop] 0/32, RunningAvgSamplesPerSec=1.

## testing the model

In [ ]:
an = []
for a  in me.children():
  an = a

from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-350M-multi")
# model = AutoModelForCausalLM.from_pretrained("Salesforce/codegen-350M-multi")

text = "def hello_world():"
input_ids = tokenizer(text, return_tensors="pt").input_ids.cuda()

# an.to_device("cpu")

generated_ids = an.generate(input_ids, max_length=128)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.7/dist-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


def hello_world():
    print('Hello World!')

hello_world()



In [ ]:
text = "%draw = function()"
input_ids = tokenizer(text, return_tensors="pt").input_ids.cuda()

# an.to_device("cpu")

generated_ids = an.generate(input_ids, max_length=128)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


%draw = function() {
	$(this).html($("#"+""*""*"""""""""""""""""""""""""""""""""""""""""""""""""""""""""
	$("#"""""""""""""""""""""""""""""""""""""""


In [ ]:
def gene(text):
  # text = "draw = function()"
  input_ids = tokenizer(text, return_tensors="pt").input_ids.cuda()

  # an.to_device("cpu")

  generated_ids = me.generate(input_ids, max_length=128)
  print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))

In [ ]:
me.train()

In [ ]:
torch.save(me, "/content/drive/MyDrive/bmo s/model.pt")

In [ ]:
gene("move_player = function()")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


move_player = function() {
    $(".move-player").removeClass("move-player");
    $(".move-player").addClass("move-player-" + $("#" + $.cookie("move_player")).val());
    $(".move-player").val($("#" + $.cookie("move_player")).val());
    $(".move-player").change(function() {
      $(".move-player").removeClass("move-player");
      $(".move-player").addClass("move-


In [ ]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-350M-multi")
# model = AutoModelForCausalLM.from_pretrained("Salesforce/codegen-350M-multi")

# text = "def hello_world():"
# input_ids = tokenizer(text, return_tensors="pt").input_ids

# generated_ids = model.generate(input_ids, max_length=128)
# print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))
